# Analyzing CIA Factbook Data Using SQL

### Genral house-keeping for executing SQL in Jupyter Notebook

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

### Getting details of db

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


### Displaying 1st 5 rows of facts table of factbook.db

In [3]:
%%sql

SELECT * FROM facts LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Minimum and Maximum Population and Population Growth

In [4]:
%%sql

SELECT MIN(population) as minimum_population,MAX(population) as maximum_population,
        MIN(population_growth) as min_population_growth,MAX(population_growth) as max_population_growth
FROM facts

 * sqlite:///factbook.db
Done.


minimum_population,maximum_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


### Countries with minimum population

In [5]:
%%sql

SELECT name FROM facts WHERE population IN (SELECT MIN(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
Antarctica


### Countries with maximum population

In [6]:
%%sql

SELECT name FROM facts WHERE population IN (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
World


##### So we see that the database has record for the whole world as well. So we must exclude them and then perform summary statistics calculation.

In [7]:
%%sql

SELECT MIN(population) as minimum_population,MAX(population) as maximum_population,
        MIN(population_growth) as min_population_growth,MAX(population_growth) as max_population_growth
FROM facts WHERE name <> "World"

 * sqlite:///factbook.db
Done.


minimum_population,maximum_population,min_population_growth,max_population_growth
0,1367485388,0.0,4.02


### Average value of population and area

In [8]:
%%sql

SELECT AVG(population),AVG(area) FROM facts

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


### Countries having population above average and area below average

In [9]:
%%sql

SELECT name FROM facts WHERE population > (SELECT AVG(population) FROM facts) AND 
                                area < (SELECT AVG(area) FROM facts)

 * sqlite:///factbook.db
Done.


name
Bangladesh
Germany
Japan
Philippines
Thailand
United Kingdom
Vietnam


## Answering some questions based on the database :

### What country has the most people? What country has the highest growth rate?

In [10]:
%%sql

SELECT name AS 'Most Populated Country' FROM facts 
            WHERE population IN (SELECT MAX(population) FROM facts WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


Most Populated Country
China


In [11]:
%%sql

SELECT name AS 'Country with Highest Growth Rate'  FROM facts 
            WHERE population_growth IN (SELECT MAX(population_growth) FROM facts)

 * sqlite:///factbook.db
Done.


Country with Highest Growth Rate
South Sudan


### Which countries have the highest ratios of water to land?

In [12]:
%%sql 

SELECT name AS 'countries with the highest ratios of water to land',CAST(area_water AS FLOAT)/area_land AS Ratio
FROM facts WHERE CAST(area_water AS FLOAT)/area_land IN (SELECT MAX(CAST(area_water AS FLOAT)/area_land) FROM facts)

 * sqlite:///factbook.db
Done.


countries with the highest ratios of water to land,Ratio
British Indian Ocean Territory,905.6666666666666


### Which countries have more water than land?

In [13]:
%%sql 

SELECT name AS 'countries with more water than land',CAST(area_water AS FLOAT)/area_land AS Ratio
FROM facts WHERE CAST(area_water AS FLOAT)/area_land > 1

 * sqlite:///factbook.db
Done.


countries with more water than land,Ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832


### Which countries will add the most people to their population next year?

In [14]:
%%sql

SELECT name AS 'Country with Highest Growth Rate',population_growth FROM facts 
            WHERE population_growth IN (SELECT MAX(population_growth) FROM facts)

 * sqlite:///factbook.db
Done.


Country with Highest Growth Rate,population_growth
South Sudan,4.02


### Which countries have a higher death rate than birth rate?

In [15]:
%%sql

SELECT name AS 'Countries with higher death rates',death_rate,birth_rate FROM facts WHERE death_rate > birth_rate

 * sqlite:///factbook.db
Done.


Countries with higher death rates,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45
Czech Republic,10.34,9.63
Estonia,12.4,10.51
Germany,11.42,8.47
Greece,11.09,8.66
Hungary,12.73,9.16


### What countries have the highest population/area ratio ?

In [16]:
%%sql

SELECT name AS 'Country Name',population/area AS Ratio FROM facts ORDER BY Ratio DESC LIMIT 10

 * sqlite:///factbook.db
Done.


Country Name,Ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


## Thank You !